In [8]:
import sys
from pathlib import Path

project_root = Path.cwd().parents[0]  # importing functions from other folders
sys.path.insert(0, str(project_root))

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from _data.data_utils import read_in
from _fitting.fitting_utils import hist_plot, CI_plot, CI_plot_alt, CI_plot_both, plot_posteriors_side_by_side, plot_spline
from _fitting.model_utils import build_model, data_settings_to_name, model_settings_to_name
import pymc as pm
import arviz as az
from patsy import dmatrix
import nutpie
import time
from pprint import pprint
from IPython.display import display
import seaborn as sns
from pymc.variational.callbacks import CheckParametersConvergence

az.style.use("arviz-darkgrid")


if '___laptop' in os.listdir('../'):
    # laptop folder
    folder = "../../_data/p-dengue/"
elif '___server' in os.listdir('../'):
    # server folder
    folder = "../../../../../data/lucaratzinger_data/p_dengue/"

In [9]:
data_settings = {'admin':2, 'max_lag':6, 'start_year':2016, 'start_month':1, 'end_year':2019, 'end_month':12}
data = read_in(folder, **data_settings, standardise=True, dropna=True, celsius=True, tp_log=True)

In [10]:
data_settings_to_name(data_settings)

'a2_201601_201912'

In [11]:
list(data.columns)

['admin1',
 'admin2',
 'year',
 'month',
 'cases',
 'ONI',
 't2m_mean_unweighted (0)',
 't2m_mean_pop_weighted (0)',
 't2m_min_unweighted (0)',
 't2m_min_pop_weighted (0)',
 't2m_max_unweighted (0)',
 't2m_max_pop_weighted (0)',
 'rh_mean_unweighted (0)',
 'rh_mean_pop_weighted (0)',
 'tp_24hmean_unweighted (0)',
 'tp_24hmean_pop_weighted (0)',
 'tp_24hmax_unweighted (0)',
 'tp_24hmax_pop_weighted (0)',
 'tp_24hmean_unweighted_log1p (0)',
 'tp_24hmean_unweighted_log (0)',
 'tp_24hmean_pop_weighted_log1p (0)',
 'tp_24hmean_pop_weighted_log (0)',
 'tp_24hmax_unweighted_log1p (0)',
 'tp_24hmax_unweighted_log (0)',
 'tp_24hmax_pop_weighted_log1p (0)',
 'tp_24hmax_pop_weighted_log (0)',
 't2m_mean_unweighted (1)',
 't2m_mean_pop_weighted (1)',
 't2m_min_unweighted (1)',
 't2m_min_pop_weighted (1)',
 't2m_max_unweighted (1)',
 't2m_max_pop_weighted (1)',
 'rh_mean_unweighted (1)',
 'rh_mean_pop_weighted (1)',
 'tp_24hmean_unweighted (1)',
 'tp_24hmean_pop_weighted (1)',
 'tp_24hmax_unweighte

In [12]:
settings = {'surveillance_name':'urban_surveillance_pop_weighted',
            'urbanisation_name':'urbanisation_pop_weighted_std',
            'stat_names':[], 'degree':3, 'num_knots':5, 'knot_type':'quantile','orthogonal':True}

In [13]:
model_dict = {}
idata = {}
idata_vi = {}

In [14]:
for surv in [None, 'surveillance_pop_weighted', 'urban_surveillance_pop_weighted']:
    settings = {
            'surveillance_name':surv,
            'urbanisation_name':None,
            'stat_names':[], 'degree':3, 'num_knots':5, 'knot_type':'quantile','orthogonal':True}
    model_dict[model_settings_to_name(settings)] = settings
    

In [15]:
model_dict

{'[nosurv nourb] [nostat] []': {'surveillance_name': None,
  'urbanisation_name': None,
  'stat_names': [],
  'degree': 3,
  'num_knots': 5,
  'knot_type': 'quantile',
  'orthogonal': True},
 '[surv_p nourb] [nostat] []': {'surveillance_name': 'surveillance_pop_weighted',
  'urbanisation_name': None,
  'stat_names': [],
  'degree': 3,
  'num_knots': 5,
  'knot_type': 'quantile',
  'orthogonal': True},
 '[urb_surv_p nourb] [nostat] []': {'surveillance_name': 'urban_surveillance_pop_weighted',
  'urbanisation_name': None,
  'stat_names': [],
  'degree': 3,
  'num_knots': 5,
  'knot_type': 'quantile',
  'orthogonal': True}}